# Introduction
Martyn has tried to model the growth of the model when grown on different sugars. We saw growth as expected for all sugars except arbutin, rhamnose, and salicin. We know experimentally the strain should grow on these sugars. So here I'll look into why the model doesn't grow on these sugars and try to fix it.

In [1]:
import cameo
import pandas as pd
import cobra.io
from cobra import Reaction, Metabolite

In [2]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

In [3]:
model_e_coli = cameo.load_model('iML1515')

## Rhamnose
In the rhamnose metabolism, the problem that prevents flux through the pathway is that in reaction RMPA L-Lactaldehyde is formed. This is currently a dead end and so this needs to be fixed to allow metabolism of rhamnose. I'll look into how this metabolite should be converted and linked to general metabolism. 

Nice paper talking about rhamnose degradation in various bacteria: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3870299/pdf/fmicb-04-00407.pdf

Here they show and confirm that B. subtilis has a bifunctional RhaEW protein, which can catalyze the L-rhamnulose-phosphate aldolase reaction as wel as be a lactaldehyde dehydrogenase. This protein is encoded by the yuxG gene in B. subtilis. Running a tBLASTn with the protein sequence, we find a significant hit in our strain. 

So the likely presence of this enzyme would mean that the lactaldehyde can be converted into lactate. Therefore I will add this reaction into the model.


In [4]:
#add reaction
model.add_reaction(Reaction(id='LALDDH', name = 'Llactaldehyde dehydrogenase'))

In [5]:
model.reactions.LALDDH.annotation['sbo'] = 'SBO:0000176'
model.reactions.LALDDH.annotation['rhea'] = '14280'
model.reactions.LALDDH.annotation['kegg.reaction'] = 'R01446'
model.reactions.LALDDH.annotation['ec-code'] = '1.2.1.22'

In [6]:
model.reactions.LALDDH.bounds = (0,1000)

In [7]:
model.reactions.LALDDH.add_metabolites({
    model.metabolites.lald__L_c:-1,
    model.metabolites.nad_c:-1,
    model.metabolites.h2o_c:-1,
    model.metabolites.lac__L_c:1,
    model.metabolites.nadh_c:1,
    model.metabolites.h_c:2
})

In [8]:
model.reactions.LALDDH.gene_reaction_rule='yuxG'

Now the problem that our strain can't grow exclusively on rhamnose is fixed. we can save & commit

In [9]:
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

## Arbutin
First I'll dig into arbutin metabolism.

Uptake into the cell is possible, via the pts system. The only following steps is via AB6PGH into G6P. This reaction produces benzenediol (hqn_c). This is a dead end metabolite and so thats why this pathway can never carry flux and our model can't grow on arbutin. 

Benzenediol is a substituted phenol. Various geobacillus species have been shown to be able to degrade phenols (see https://doi.org/10.1016/bs.aambs.2015.03.001 and doi:10.1016/j.jprot.2011.10.035 and http://production.datastore.cvt.dk/filestore?oid=5490af28bde7fcd94f4f839b&targetid=539ef0440370e0c1720065d8 and  doi:10.1186/1471-2180-8-197). 

Thermophilic bacilli have been shown to generally use the meta-pathway for metabolizing phenol (see the papers above). This consists of hydroxylation of the aromatic to catechol, followed by ring cleavage by the catechol-2,3-dioxygenase and then a final oxidation or hydrolysis. The enzymes known to catabolize phenol are also known to react with substitued phenols. So it would be most expected that this pathway is present in our strain too, if it can grow on arbutin. These steps should then be able to convert the benzenediol with a similar mechanism as phenol. 

The first enzyme of the proposed pathway, phenol hydrozylase (PheA) has been confirmed in G. thermodenitrificans. Also both subunits of the enzyme are annotated in our strain. This reaction catalyzes the conversion of the phenol into the correspnoding catechol by adding a -OH group.

The next step in the degradation is the catechol 2,3 dioxygenase. This enzyme is also annotated in our strain. Finally, our strain has a hydrolase annotated, as well as the following downstream steps that are involved in converting catechol into the final pyruvate and acetyl-Coa. 

All this taken together points to the likelihood that this pathway is responsible for the metabolism of the phenols that are produced when the arbutin (or other sugars) are metabolized. Especially considering the evidence that this pathway works on a wide rande of substituted phenols. I will therefore make sure this is implemented into our model.

One issue is that the pathway is not completely studied, also considering the fact that the enzymes can react with many different substituted phenols. Therefore there is no evidence for our exact molecules how it would progress through the pathway, but based on reaction mechanism I will estimate and predict what the molecules are that are formed throughout the pathway.

For Benzenediol, up until the hydroxymuconate semialdehyde, the reactions would run similarly to the pathway as known. But after the reaction catalyzed by the 	2-hydroxymuconate-semialdehyde hydrolase the resulting molecule is already 4-hydroxy-2-oxopent-4-enoate. the hydration by the 2-keto-4-pentenoate hydratase is then no longer needed, and the 4-hydroxy-2-oxopent-4-enoate can be metabolized into acetyl-coA and pyruvate in the same way the phenol would be. 

I will not incorporate these steps into the model.

In [4]:
#step one: phenol hydroxylase
#add metabolite
model.add_metabolites(Metabolite(id='hql_c', name = 'hydroxyquinol', compartment = 'c', charge = 0, formula = 'C6H6O3'))

In [5]:
model.metabolites.hql_c.annotation['sbo'] = 'SBO:0000247'
model.metabolites.hql_c.annotation['kegg.compound'] = 'C02814'
model.metabolites.hql_c.annotation['chebi'] = 'CHEBI:16971'
model.metabolites.hql_c.annotation['metanetx.chemical'] = 'MNXM722'

In [6]:
#add phenol hydratase reaction
model.add_reaction(Reaction(id='PHEHY', name = 'Phenol hydratase'))

In [7]:
model.reactions.PHEHY.annotation['sbo'] = 'SBO:0000176'
model.reactions.PHEHY.annotation['kegg.reaction'] = 'R00815'
model.reactions.PHEHY.annotation['ec-code'] = '1.14.13.7'
model.reactions.PHEHY.annotation['rhea'] = '17064'

In [8]:
model.reactions.PHEHY.bounds = (0,1000)

In [9]:
model.reactions.PHEHY.add_metabolites({
    model.metabolites.hqn_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.o2_c:-1,
    model.metabolites.hql_c:1,
    model.metabolites.nadp_c:1,
    model.metabolites.h2o_c:1,
    model.metabolites.h_c:-1
})

In [10]:
#step two: catechol 2,3 dioxygenase
# first add metabolite
model.add_metabolites(Metabolite(id='dhmsa_c', name = '2,4-dihydroxymuconate semialdehyde', compartment = 'c', charge = 0, formula = 'C6H6O5'))

In [11]:
model.metabolites.dhmsa_c.annotation['sbo'] = 'SBO:0000247'

In [12]:
#add catechol 2,3 dioxygenase reaction with the hydroquinol substrate
model.add_reaction(Reaction(id='C23OHQL', name = 'catechol 2,3 dioxygenase'))

In [13]:
model.reactions.C23OHQL.annotation['sbo'] = 'SBO:0000176'
model.reactions.C23OHQL.annotation['ec-code'] = '1.13.11.2'

In [14]:
model.reactions.C23OHQL.bounds = (0,1000)

In [15]:
model.reactions.C23OHQL.add_metabolites({
    model.metabolites.hql_c:-1,
    model.metabolites.o2_c:-1,
    model.metabolites.dhmsa_c:1
})

In [16]:
#next step: 2-hydroxymuconate-6-semialdehyde hydrolase
#metabolite is already present: 4h2opntn_c
#add reaction
model.add_reaction(Reaction(id='HMSAH', name = '2-hydroxymuconate-6-semialdehyde hydrolase'))

In [17]:
model.reactions.HMSAH.annotation['sbo'] = 'SBO:0000176'
model.reactions.HMSAH.annotation['ec-code'] = '3.7.1.9'

In [18]:
model.reactions.HMSAH.bounds = (0,1000)

In [19]:
model.reactions.HMSAH.add_metabolites({
    model.metabolites.dhmsa_c:-1,
    model.metabolites.for_c:1,
    model.metabolites.get_by_id('4h2opntn_c'):1,
    model.metabolites.h2o_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1
})

Here we make the assumption that this reaction requires NADPH, as it requires the input of reducing power, which is generally done by NADPH.

The formed 4h2opntn_c metabolite should be able to be further catabolized into pyruvate and acetyl-coa. This is done consecutively by the HOPNTAL and ACALD reactions. So now our model is able to grow on arbutin!

In [22]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

In [21]:
#check that the model can then grow
with model:
    model.reactions.EX_glc__D_e.bounds = (0,0)
    model.reactions.EX_arbt_e.bounds = (-10,0)
    print (model.optimize().objective_value)

1.334363563054286


# Salicin (salcn)
Our organism should grow on salicin, but current our model doesn't. I'll investigate what the problem is here and try to fix it in the correct way so our model can grow on this sugar. 

The salicin is taken up by PTS and can then the salicin-6-phosphate can be converted into g6p and 2hymeph_c. The 2hymeph_c then reacts with udpg to give salicin again. The problem here is that to be able to reconvert the 2hymeph, youi need the glucose that is formed when the 2hymeph is formed, so in the end there is no sugar avaiable for growth. This is ofcourse not correct and so will be investigated and fixed.

the 2hymeph is a phenol. So here the same problem applies as with arbutin growth. Therefore, I will try to 


In [13]:
model.metabolites.get_by_id('2hymeph_c').name = 'Salicyl alcohol'

In [8]:
model.reactions.SALCN6PH.annotation

{'sbo': 'SBO:0000176',
 'ec-code': '3.2.1.86',
 'kegg.reaction': 'R05134',
 'metanetx.reaction': 'MNXR109345',
 'rhea': '33127'}

In [15]:
model.metabolites.get_by_id('2hymeph_c').annotation

{'sbo': 'SBO:0000247',
 'chebi': 'CHEBI:16464',
 'kegg.compound': 'C02323',
 'metanetx.chemical': 'MNXM1855'}

In [15]:
#check that the model can then grow
with model:
    model.reactions.EX_glc__D_e.bounds = (0,0)
    model.reactions.EX_arbt_e.bounds = (-10,0)
    print (model.optimize().objective_value)

-0.008875098013070872


C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\util\solver.py:416: UserWarning:

solver status is 'infeasible'



In [21]:
model.reactions.AB6PGH

Reaction identifier,AB6PGH
Name,Arbutin 6-phosphate glucohydrolase
Memory address,0x028762c46bc8
Stoichiometry,arbt6p_c + h2o_c + h_c --> g6p_c + hqn_c Arbutin 6-phosphate + Water + H+ --> Glucose 6-phosphate + p-Benzenediol
GPR,RTMO03636 or RTMO02650
Lower bound,0.0
Upper bound,1000.0


In [22]:
model.reactions.AB6PGH.annotation

{'sbo': 'SBO:0000176',
 'ec-code': '3.2.1.86',
 'kegg.reaction': 'R05133',
 'metanetx.reaction': 'MNXR109344',
 'rhea': '28510'}